In [28]:
#Desafío - Transformación y manipulación de datos (Parte II)

#0. Importar librería:

import pandas as pd

#1. Cargar los datos de cada pestaña del archivo "US_Regional_Sales_Data.xlsx" en un DataFrame independiente:
archivo = pd.ExcelFile("US_Regional_Sales_Data.xlsx")

df_sales_orders = pd.read_excel(archivo, "Sales Orders Sheet")
df_customers = pd.read_excel(archivo, "Customers Sheet")
df_store_locations = pd.read_excel(archivo, "Store Locations Sheet")
df_products = pd.read_excel(archivo, "Products Sheet")
df_sales_team = pd.read_excel(archivo, "Sales Team Sheet")

In [29]:
#2. Cruzar todos los DataFrames usando validación many_to_one, y guardar el resultado en una nueva variable llamada "df_base":
df_base = pd.merge(df_sales_orders, df_customers, on="_CustomerID", how="left")
df_base = pd.merge(df_base, df_store_locations, on="_StoreID", how="left")
df_base = pd.merge(df_base, df_products, on="_ProductID", how="left")
df_base = pd.merge(df_base, df_sales_team, on="_SalesTeamID", how="left")

In [76]:
#3. Agregar las columnas de la tabla a "df_base":

#Asegurar que sean de tipo datetime:
df_base["ProcuredDate"] = pd.to_datetime(df_base["ProcuredDate"])
df_base["OrderDate"] = pd.to_datetime(df_base["OrderDate"])
df_base["ShipDate"] = pd.to_datetime(df_base["ShipDate"])
df_base["DeliveryDate"] = pd.to_datetime(df_base["DeliveryDate"])

#Crear las columnas solicitadas:
df_base["ProcurementDays"] = (df_base["OrderDate"] - df_base["ProcuredDate"]).dt.days
df_base["ShippingDays"] = (df_base["ShipDate"] - df_base["OrderDate"]).dt.days
df_base["DeliveryDays"] = (df_base["DeliveryDate"] - df_base["ShipDate"]).dt.days
df_base["CustomerDays"] = df_base["ShippingDays"] + df_base["DeliveryDays"]

In [80]:
#4. Agregar una nueva columna discreta al "df_base", con el nombre "CustomerDaysInterval" que clasifique los valores de la
#columna "CustomerDays" en los intervalos señalados:
df_base["CustomerDaysInterval"] = pd.cut(df_base["CustomerDays"],
                                          bins=[0, 15, 30, 45, 60, 75, 90],
                                          labels=["0 to 15 days", "15 to 30 days", "30 to 45 days",
                                                  "45 to 60 days", "60 to 75 days", "75 to 90 days"])

In [107]:
#5. Utilizar una tabla pivote para mostrar el conteo de órdenes agrupando por "Sales Team" en las filas y "CustomerDaysInterval" en las columnas:
tabla_pivote_conteo = pd.pivot_table(df_base, values ="OrderNumber", index ="Sales Team", columns="CustomerDaysInterval", aggfunc="count", fill_value = 0)

print(tabla_pivote_conteo)

CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
Sales Team                                                         
Adam Hernandez                  81            179             42   
Anthony Berry                   76            181             41   
Anthony Torres                  81            162             45   
Carl Nguyen                     96            178             40   
Carlos Miller                   65            148             34   
Chris Armstrong                 92            156             44   
Donald Reynolds                 97            163             36   
Douglas Tucker                  87            145             45   
Frank Brown                     85            150             40   
George Lewis                   104            169             42   
Jerry Green                     85            165             46   
Joe Price                       87            138             41   
Jonathan Hawkins                81            14

In [95]:
#6. Agregar la columna calculada "GrossMargin" a la base:
df_base["GrossMargin"] = df_base["Order Quantity"] * (df_base["Unit Price"] * (1 - df_base["Discount Applied"]) - df_base["Unit Cost"])

In [112]:
#7. Aplicar un porcentaje a la columna "GrossMargin", pero ese porcentaje dependerá del intervalo en que se encuentre GrossMargin:
df_base["CommissionsPercentage"] = pd.cut(df_base["GrossMargin"],
                                           bins=[-float("inf"), 100, 1000, 10000, 100000, float("inf")],
                                           labels=[0.05, 0.1, 0.15, 0.2, 0.0]).astype(float)

In [99]:
#8. Calcular el monto de la comisión en la columna "CommissionsAmount":
df_base["CommissionsAmount"] = df_base["GrossMargin"] * df_base["CommissionsPercentage"]

In [101]:
#9. Calcular la comisión sobre el margen bruto en la columna "NetMargin":
df_base["NetMargin"] = df_base["GrossMargin"] - df_base["CommissionsAmount"]

In [108]:
#10. Utilizando una tabla pivote, mostrar la suma de los valores "GrossMargin", "NetMargin" y "CommissionAmount" agrupando por "Sales Team" en las filas:
tabla_pivote_suma = pd.pivot_table(df_base, values = ["GrossMargin", "NetMargin", "CommissionsAmount"], index = "Sales Team", aggfunc = "sum")

print(tabla_pivote_suma)

                     CommissionsAmount  GrossMargin      NetMargin
Sales Team                                                        
Adam Hernandez           125360.740275  779095.1955  653734.455225
Anthony Berry            123515.402825  767943.9500  644428.547175
Anthony Torres           117421.270425  743901.4355  626480.165075
Carl Nguyen              123296.388200  774781.4675  651485.079300
Carlos Miller            109759.334675  668752.9290  558993.594325
Chris Armstrong          122474.986625  781608.7005  659133.713875
Donald Reynolds          143147.349200  885386.8095  742239.460300
Douglas Tucker           120353.208850  745528.5640  625175.355150
Frank Brown              104637.715650  670881.2510  566243.535350
George Lewis             136425.941025  839150.7460  702724.804975
Jerry Green              126237.850675  788587.7555  662349.904825
Joe Price                128634.050400  787707.4425  659073.392100
Jonathan Hawkins         125744.182950  781391.5870  655647.40